In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**# Analyis On the Olympics Data, on Four Thematic Questions.**

* What effect does the host country has on the medals won.
* Is the economic factors GDP of countries affect the medal winning.
* Is the age of winning olympics changing.
* Review of Kenya contrbution to Olympics.



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('darkgrid')

In [ ]:
athletes_data = pd.read_csv('../input/120-years-of-olympic-history/athlete_events.csv');
countries_data = pd.read_csv('../input/120-years-of-olympic-history/country_definitions.csv');
word_cities = pd.read_csv('../input/world-cities-datasets/worldcities.csv');
gdp_data = pd.read_csv('../input/global-gdp-per-capita-ppp-from-1800-to-2040/Global GDP per capita PPP from 1800 to 2018 with projections to 2040.csv');

In [ ]:
df = athletes_data.merge(countries_data, how = 'left', on = 'NOC')

In [ ]:
countries_data.duplicated().sum()

In [ ]:
#make column names consistent
df.rename(columns = {'region': 'Region', 'notes' : 'Notes'}, inplace = True)
df.head()
#https://jman4190.medium.com/explaining-the-inplace-parameter-for-beginners-5de7ffa18d2e#:~:text=When%20inplace%20%3D%20True%20%2C%20the%20data,to%20save%20it%20to%20something.

In [ ]:
#TOP 10 COUNTRIES BY PARTICIPATION.

top_10_countries = df.Team.value_counts().sort_values(ascending=False).head(10)
#top_10_countries
plt.figure(figsize = (12,6))
plt.title('Top 10 Countries By Participation Global')
sns.barplot(x=top_10_countries.index, y=top_10_countries, palette = 'Set2');

In [ ]:
# AGE DISTRIBUTION OF PARTICIPANTS.
plt.figure(figsize=(12,6))
plt.title('Partcipants Age Distribution')
plt.xlabel('Age')
plt.ylabel('Number of Participants')
plt.hist(df.Age, bins = np.arange(10, 80, 5), color = 'SkyBlue', edgecolor = 'white');

In [ ]:
age_data = df.groupby(['Region','Age'], as_index = False).agg(
                                                counts = pd.NamedAgg(column = 'ID', aggfunc = 'count'))
age_data.sort_values(by = 'Age', ascending=True) 

* Mean age of participants is 25 years of age, 

* Youngest participnt was a ten year old from Greece:  Name:Dimitrios Loundras    Country: Greece    Year: 1896 Sport: Gymnastics.
 
* The oldest participants were 96 and 97 from USA both Male doing art competitions.

In [ ]:
# Pie Plot for Male and Female Participants
gender_counts = df.Sex.value_counts()
#gender_counts
plt.figure(figsize=(12,6))
plt.title('Partcipants By Gender')
plt.pie(gender_counts, labels =  gender_counts.index, autopct = '%1.1f%%', explode=[0.05]*2, 
                                        startangle = 150,pctdistance=0.5, shadow = True);

In [ ]:
# Participations Trend By Gender for Winter Olympics
#women_athletes = df[(df.Sex == 'F') & (df.Season == 'Winter')]
gender_participants = df[df.Season == 'Winter']
part = gender_participants.groupby('Year')['Sex'].value_counts()

plt.figure(figsize=(20,10))
part.loc[:, 'M'].plot(label = 'Male')
part.loc[:, 'F'].plot(label = 'Female')
plt.legend(loc="upper left")
plt.title('Male and Female Participants Count Trend For Winter Olympics');

In [ ]:
# Participations Trend By Gender for Summer Olympics

gender_participants = df[df.Season == 'Summer']
part = gender_participants.groupby('Year')['Sex'].value_counts()

plt.figure(figsize=(20,10))
part.loc[:, 'M'].plot(label = 'Male')
part.loc[:, 'F'].plot(label = 'Female')
plt.legend(loc="upper left")
plt.title('Male and Female Participants Count Trend For Summer Olympics');

In [ ]:
#Weight Vs Height of Olympics Participants By Gender

plt.figure(figsize = (12,10))
axis = sns.scatterplot(x='Height', y = 'Weight', data = df, hue = 'Sex')
plt.title('Weight Vs Height of Olympics Participants By Gender')

Generally male participants are taller and heavy compared to female counter parts shorter and lighter alluled by Scatter Plot above.

# Thematic Question 1

What effect does the host country has on the medals won.

In [ ]:
#https://www.youtube.com/watch?v=q1FttL_G1G4
#https://towardsdatascience.com/visual-analysis-of-olympics-data-16273f7c6cf2
#https://www.youtube.com/watch?v=5X5LWcLtkzg

word_cities.rename(columns = {'city':'City', 'country': 'Host_Country'}, inplace=True)
ct = word_cities[['Host_Country', 'City']]
#ct.duplicated().sum()
x = ct.drop_duplicates()
#x.duplicated().sum()
y = x.drop_duplicates(subset=['Host_Country'])

data = df.merge(y, how = 'inner', on = 'City')
data.head()

In [ ]:
plt.figure(figsize=(10,5))
plt.tight_layout()
#sns.countplot(data['Host_Country'])
data['Host_Country'].value_counts().plot(kind="bar")
plt.title('Participation Per Host Country')
plt.xticks(rotation=45)

In [ ]:
has_medals = data[data.Medal.isin (['Gold', 'Bronze', 'Silver'])]
medals = has_medals.Host_Country.value_counts().reset_index(name = 'Medal')

g = sns.catplot(x = 'index', y = 'Medal', data = medals, height = 10, 
                                            kind = 'bar', palette = 'rocket' )
g.despine(left = True)
g.set_xlabels('Host Country')
g.set_ylabels('Medals Won')
plt.title('Medals Won Per Host Country')
g.set_xticklabels(rotation=60)
#plt.xticks(rotation=45)

In [ ]:
#https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/

md = has_medals[['Year', 'Host_Country', 'Medal']]
#m = md.groupby(['Year', 'Host_Country'])['Medal'].count().reset_index(name = 'Medals_Count')
#m = md.groupby('Host_Country', as_index = False).agg({'Medal':"count",    'Year': "nunique" })

m = md.groupby('Host_Country', as_index = False).agg(
                                                No_Medals = pd.NamedAgg(column = 'Medal', aggfunc ='count'),
                                                No_Hosts = pd.NamedAgg(column = 'Year',  aggfunc = 'nunique')
)
m = m.sort_values(by = 'No_Medals', ascending = False)
                            

In [ ]:
fig = plt.figure(figsize = (20,10))
ax1 = plt.subplot(1,1,1)
ax1.bar(m['Host_Country'], m['No_Medals'])
ax1.set_ylim(0, m['No_Medals'].max()*1.25)
ax1.yaxis.grid(linestyle = '-', linewidth = 0.5)
ax1.set_axisbelow(True)
#ax1.spines['top'].set_linewidth(0)
ax2 = ax1.twinx()
ax2.plot(m['Host_Country'], m['No_Hosts'], 'o-', color = '#cf6a63', linewidth = 3)
ax2.set_ylim(0, m['No_Hosts'].max()*1.25)
plt.title('Corelation Between Medals Winning and Number of Hosts', fontsize = 18, weight = 'bold')
#plt.xlabel("Host Country", size = 15)
plt.show()

* Its obvious that hosting has a huge contribution to medals won, without going to the nitty grittys of it.United Kingdom, followed by Australia had the highest numbers of participants and medals won consecutively.

* United Kingdom as per the data had the highest hosts followed by France, this insight is erronoues factually but hypothetically from data is right.

In [ ]:
md = has_medals[['Year', 'Region', 'Medal', 'Season']].sort_values(by = 'Year', ascending=True) 
md = md.query('Region == "France" and Season == "Summer"')
md = md[['Year', 'Region']]
xt = md.groupby('Year')['Year'].count()


plt.figure(figsize=(20,10))
xt.plot()
plt.title('Medals Won By France Summer Season');
#https://towardsdatascience.com/data-grouping-in-python-d64f1203f8d3
#https://www.geeksforgeeks.org/python-filtering-data-with-pandas-query-method/

* In case of france above the years when France hosted Summer olympics 1900 and 1924 respectively medals won tremendsouly increased in those areas

# Thematic Question 2:

**Does the GDP of a country affect Medal wining?**

In [ ]:
md = has_medals[['Year', 'Region', 'Medal', 'Season']].sort_values(by = 'Year', ascending=True) 
md.rename(columns = {'Region': 'Country', 'GDP per capita PPP' : 'GDP'}, inplace = True)
gxt = md.groupby(['Year','Country'])['Year'].count().reset_index(name = 'Medals_Count')

In [ ]:
gdp = gdp_data[['Year', 'Country', 'GDP per capita PPP']]

In [ ]:
gdp.rename(columns = {'GDP per capita PPP' : 'GDP'}, inplace = True)

In [ ]:
has_medals_with_gdp = gxt.merge(gdp, how = 'inner', on = ['Country', 'Year'])
has_medals_with_gdp

In [ ]:
plt.figure(figsize = (12,10))

axis = sns.scatterplot(x='GDP', y = 'Medals_Count', data = has_medals_with_gdp)
plt.title('Relationship Between Medals Winning and GDP')

In [ ]:
plt.figure(figsize = (18,12))
sns.heatmap(has_medals_with_gdp[['Medals_Count', 'GDP']].corr(),annot = True,cmap = 'Greens')
plt.title("Corelation Between Medals Winning and GDP")

* There do exist corelation between medal winning and GDP  0.1

In [ ]:
#Corelation For Medals won by Kenyans and Kenya GDP.
kenya = has_medals_with_gdp.query('Year > 1960 and Country == "Kenya" ').reset_index()

In [ ]:
fig = plt.figure(figsize = (16,10))
ax1 = plt.subplot(1,1,1)
ax1.bar(kenya['Year'][kenya['Medals_Count'] > 0], kenya['Medals_Count'])
ax1.set_ylim(0, kenya['Medals_Count'].max()*1.25)
ax1.yaxis.grid(linestyle = '-', linewidth = 0.5)
ax1.set_axisbelow(True)
#ax1.spines['top'].set_linewidth(0)
ax2 = ax1.twinx()
ax2.plot(kenya['Year'], kenya['GDP'], 'o-', color = '#cf6a63', linewidth = 3)
ax2.set_ylim(0, kenya['GDP'].max()*1.25)
plt.title('Corelation Between Medals Winning and GDP Kenya', fontsize = 18, weight = 'bold')
plt.show()

* There exist Corelation between GDP and Medal winning for Kenya

# Thematic Question 3: Is the Age of winnning Olympics Medals Changing?

In [ ]:
age_medals_winning = has_medals.groupby(['Year', 'Sex'])['Age'].mean().reset_index(name = 'Mean_Age')

male_age_medals = age_medals_winning[['Year', 'Mean_Age']][age_medals_winning['Sex'] == 'M']
female_age_medals = age_medals_winning[['Year', 'Mean_Age']][age_medals_winning['Sex'] == 'F']

In [ ]:
fig = plt.figure(figsize = (16,10))
ax = plt.axes()
ax.plot(male_age_medals['Year'],male_age_medals['Mean_Age'], label = 'Male', color = '#99011F')
ax.plot(female_age_medals['Year'],female_age_medals['Mean_Age'], label = 'Female', color = 'orange')
ax.set_ylim(20, 38)
plt.axis('tight')
plt.xlabel('Olympics Year')
plt.ylabel('Mean Age')
plt.title('Male and Female Mean Age of Winning Olympics Medals')
plt.legend();

* The average medal winning mean age has been changing from olympics year to another, lowest mean age for females was 21 years in 1924 and highest 32 years in 1904.
 
* Average Min medal winning age for males was 25 years in 1984 and highest was 35.8 years in 1954.

* The average medal winning age stabilized from 2000 onwards at 25.7 years and 26.3 years for females and males respectively.

# Kenya Contribution to Olympics

In [ ]:
kenya = data[['Name','Sex','Year', 'Season', 'Sport', 'Medal', 'Team']].query('Team =="Kenya" ')

In [ ]:
kenya_participants = kenya.groupby(['Year', 'Sex', 'Medal'])['Name'].count().reset_index()
kenya_participants

In [ ]:
part = kenya.groupby('Year')['Sex'].value_counts()
plt.figure(figsize=(20,10))
part.loc[:, 'M'].plot(label = 'Male')
part.loc[:, 'F'].plot(label = 'Female')
plt.legend(loc="upper left")
plt.title('Male and Female Participants Count Trend For Summer Olympics');

* Since 1964 to date Kenya has only participated on Olympics Summer games.
* At 1964 had male representation only (41) in number, thats immedetialy after independence, and **Wilson Arap Chuma Kiprugut, Athletee** won a Bronze Medal.
* First Female participation was in 1968, 3 female respensentatives all athletees and no one of them won a Medal.
* First Kenyan Female **(Joyce Chepchumba Koech, Athletee)** won a Bronze Medal on 2000 Summer Olympics.

In [ ]:
kenya = data[['Name','Sex','Year', 'Season', 'Sport', 'Medal', 'Team']].query('Team =="Kenya" and Medal in ("Bronze", "Silver", "Gold")')

In [ ]:
ax = sns.histplot(
    kenya_participants,
    x='Year',
    weights='Name',
    hue='Medal',
    multiple='stack',
    palette=['#00a4ccff', '#d34f73ff', '#006747ff'],
    edgecolor='white',
    shrink=0.8
)

ax.set_title('Medals Won By Year')
sns.set(rc={'figure.figsize':(15,10)})
ax.set_ylabel('Number of Medals')

In [ ]:
ax = sns.histplot(
    kenya_participants,
    x='Year',
    weights='Name',
    hue='Sex',
    multiple='stack',
    palette=['#006747ff', '#d34f73ff'],
    edgecolor='white',
    shrink=0.8
)

ax.set_title('Medals Won By Year By Gender')
sns.set(rc={'figure.figsize':(15,10)})
ax.set_ylabel('Number of Medals')

In [ ]:
# output = []
# for x in data['Region']:
#     if x not in output:
#         output.append(x)
# print(output)